需要注意的是，式子里含exp，当一个值很大时，有可能inf，注意到加上常数，值不变，一般减去最大的数，使值更稳定

In [ ]:
import torch
from torch import Tensor
from jaxtyping import Float

def run_softmax(in_features: Float[Tensor, " ..."], dim: int) -> Float[Tensor, " ..."]:
    """
    给定输入张量，返回对输入的给定 dim 应用 softmax 的输出。
    
    Softmax 函数将任意实数向量转换为概率分布，输出值在 (0,1) 范围内且和为1。
    数学公式: softmax(x_i) = exp(x_i) / Σ exp(x_j)
    
    为了数值稳定性，使用了减去最大值的技巧，避免指数运算溢出。

    参数:
        in_features (Float[Tensor, "..."]): 要 softmax 的输入特征。形状任意。
                                           通常是注意力得分、分类logits等
        dim (int): 要应用 softmax 的 in_features 的维度。
                  对该维度进行归一化，使其和为1

    返回:
        Float[Tensor, "..."]: 与 in_features 形状相同的张量，包含对指定 dim
                             进行 softmax 归一化的输出。每个位置的值在(0,1)之间，
                             沿指定维度的和为1。
    """
    
    # 1. 数值稳定性技巧：减去每个样本在指定维度上的最大值
    # 目的：防止 exp(x) 计算时出现数值溢出
    # 数学上：softmax(x) = softmax(x - c)，其中 c 是任意常数
    # 选择 c = max(x) 可以确保所有指数都 ≤ 1，避免溢出
    dim_max = torch.amax(in_features, dim=dim, keepdim=True)
    #         ↑                                ↑
    #     取最大值                        保持维度，便于广播
    
    # 2. 减去最大值并计算指数
    # in_features - dim_max: 广播减法，每个元素减去其所在"切片"的最大值
    # 结果：最大的元素变为0，其他元素都是负数，exp后不会溢出
    dim_exp = torch.exp(in_features - dim_max)
    #         ↑
    #    指数函数，输出都在 (0, 1] 范围内
    
    # 3. 计算指定维度上的指数和
    # 这是 softmax 分母：Σ exp(x_j - max)
    sum_dim_exp = torch.sum(dim_exp, dim=dim, keepdim=True)
    #                                        ↑
    #                                  保持维度，便于除法广播
    
    # 4. 最终的 softmax 计算
    # dim_exp / sum_dim_exp: 每个指数值除以总和，得到概率分布
    # 结果：每个元素在 (0,1) 之间，沿指定维度求和等于1
    return dim_exp / sum_dim_exp

# 更详细的分步解释和示例
def softmax_step_by_step_demo():
    """演示 softmax 计算的每一步，帮助理解"""
    
    # 创建示例输入：注意力得分矩阵
    # 形状: (batch=2, heads=1, seq_len=3, seq_len=3)
    # 表示序列中每个位置对其他位置的注意力得分
    attention_scores = torch.tensor([
        [[[1.0, 2.0, 3.0],      # 第1个token对其他token的得分
          [4.0, 5.0, 6.0],      # 第2个token对其他token的得分  
          [7.0, 8.0, 9.0]]],    # 第3个token对其他token的得分
        
        [[[2.0, 1.0, 3.0],
          [5.0, 4.0, 6.0],
          [8.0, 7.0, 9.0]]]
    ])
    
    print("原始注意力得分:")
    print(attention_scores)
    print(f"形状: {attention_scores.shape}")
    
    # 对最后一个维度(每行)应用 softmax
    dim = -1  # 对每行进行归一化
    
    print(f"\n步骤1: 计算每行的最大值 (dim={dim})")
    dim_max = torch.amax(attention_scores, dim=dim, keepdim=True)
    print(f"最大值: {dim_max}")
    print(f"形状: {dim_max.shape}")
    
    print("\n步骤2: 减去最大值")
    shifted = attention_scores - dim_max
    print(f"减去最大值后: {shifted}")
    print("观察: 每行的最大值都变成了0，避免exp溢出")
    
    print("\n步骤3: 计算指数")
    dim_exp = torch.exp(shifted)
    print(f"指数值: {dim_exp}")
    print("观察: 所有值都在(0,1]范围内")
    
    print("\n步骤4: 计算每行的指数和")
    sum_dim_exp = torch.sum(dim_exp, dim=dim, keepdim=True)
    print(f"每行指数和: {sum_dim_exp}")
    
    print("\n步骤5: 归一化得到概率")
    softmax_output = dim_exp / sum_dim_exp
    print(f"softmax输出: {softmax_output}")
    
    # 验证性质
    print("\n验证 softmax 性质:")
    row_sums = torch.sum(softmax_output, dim=dim)
    print(f"每行和: {row_sums}")
    print(f"是否接近1: {torch.allclose(row_sums, torch.ones_like(row_sums))}")
    
    print(f"值范围: [{softmax_output.min():.6f}, {softmax_output.max():.6f}]")
    print("✓ 所有值都在(0,1)之间，每行和为1")

def compare_with_pytorch_builtin():
    """与 PyTorch 内置 softmax 对比，验证实现正确性"""
    
    # 测试数据
    x = torch.randn(3, 4, 5)  # 随机张量
    dim = -1  # 对最后一维应用 softmax
    
    # 我们的实现
    our_result = run_softmax(x, dim)
    
    # PyTorch 内置实现
    pytorch_result = torch.softmax(x, dim=dim)
    
    # 对比结果
    print("实现正确性验证:")
    print(f"结果是否一致: {torch.allclose(our_result, pytorch_result)}")
    print(f"最大差异: {(our_result - pytorch_result).abs().max():.2e}")

def numerical_stability_demo():
    """演示数值稳定性的重要性"""
    
    # 创建可能导致溢出的大数值
    large_values = torch.tensor([[100.0, 101.0, 102.0],
                                [1000.0, 1001.0, 1002.0]])
    
    print("数值稳定性演示:")
    print(f"输入值: {large_values}")
    
    # 不稳定的实现（直接计算指数）
    print("\n❌ 不稳定实现:")
    try:
        exp_direct = torch.exp(large_values)
        print(f"直接exp结果: {exp_direct}")
        if torch.isinf(exp_direct).any():
            print("⚠️  出现无穷大，数值溢出!")
    except:
        print("计算失败!")
    
    # 稳定的实现（我们的方法）
    print("\n✅ 稳定实现:")
    stable_result = run_softmax(large_values, dim=-1)
    print(f"稳定softmax结果: {stable_result}")
    print(f"每行和: {stable_result.sum(dim=-1)}")
    print("✓ 计算成功，结果正确!")

def attention_softmax_example():
    """展示 softmax 在注意力机制中的应用"""
    
    print("注意力机制中的 softmax 应用:")
    
    # 模拟查询和键的点积得分
    # 假设: seq_len=4, d_k=8
    query = torch.randn(1, 4, 8)    # (batch, seq_len, d_k)
    key = torch.randn(1, 4, 8)      # (batch, seq_len, d_k)
    
    # 计算注意力得分 (Q·K^T)
    attention_scores = torch.bmm(query, key.transpose(-2, -1))  # (1, 4, 4)
    print(f"注意力得分形状: {attention_scores.shape}")
    print(f"注意力得分:\n{attention_scores[0]}")
    
    # 应用 softmax 得到注意力权重
    attention_weights = run_softmax(attention_scores, dim=-1)
    print(f"\n注意力权重:\n{attention_weights[0]}")
    
    # 验证每行和为1
    row_sums = attention_weights.sum(dim=-1)
    print(f"\n每行权重和: {row_sums[0]}")
    print("✓ 每个查询的注意力权重构成概率分布")

# 使用示例
if __name__ == "__main__":
    print("=== Softmax 实现详解 ===\n")
    
    # 基本演示
    softmax_step_by_step_demo()
    
    print("\n" + "="*50 + "\n")
    
    # 正确性验证
    compare_with_pytorch_builtin()
    
    print("\n" + "="*50 + "\n")
    
    # 数值稳定性
    numerical_stability_demo()
    
    print("\n" + "="*50 + "\n")
    
    # 实际应用
    attention_softmax_example()